In [1]:
import os
import clickhouse_connect
from dotenv import load_dotenv


# qdrant_client = QdrantClient("http://127.0.0.1:6333")

load_dotenv()

ch_client = clickhouse_connect.get_client(
    host=os.getenv("CLICKHOUSE_HOST"),
    port=int(os.getenv("CLICKHOUSE_PORT")),
    username=os.getenv("CLICKHOUSE_USERNAME"),
    password=os.getenv("CLICKHOUSE_PASSWORD"),
)


In [ ]:
from emcache.base import BaseCachedEmbedding
from qdrant_client import QdrantClient

from qdrant_client.models import PointStruct, Distance, VectorParams
from qdrant_client.http.models import QueryResponse, CollectionInfo


class VectorStoreEmbedding:
    def __init__(
        self,
        backend,
        collection_prefix: str,
        qdrant_url: str = "http://127.0.0.1:6333",
        distance: Distance = Distance.COSINE,
    ):
        self.backend = backend
        self.embedder = BaseCachedEmbedding(backend)
        self.qdrant = QdrantClient(url=qdrant_url)

        # Unique collection name per model/backend
        self.collection_name = f"{collection_prefix}_{self.backend.identifier()}"

        # Create collection if needed
        self._ensure_collection(distance)

    def _ensure_collection(self, distance: Distance):
        if self.qdrant.collection_exists(self.collection_name):
            # Delete existing collection for fresh start
            print(f"🧩 Deleting existing Qdrant collection: {self.collection_name}")
            self.qdrant.delete_collection(self.collection_name)

        print(f"🧩 Creating Qdrant collection: {self.collection_name}")
        self.qdrant.create_collection(
            collection_name=self.collection_name,
            vectors_config=VectorParams(
                size=self.embedder.embed_dim,
                distance=distance,
            ),
        )

    # -----------------------------
    # Insert / Upsert
    # -----------------------------
    def add_texts(
        self,
        texts: list[str],
        payloads: list[dict] | None = None,
    ) -> list[str]:
        """Embeds and inserts new texts into Qdrant."""
        if payloads is None:
            payloads = [{} for _ in texts]

        embed_results = self.embedder.embed(texts)
        embeddings = embed_results.embedding.cpu().tolist()
        ids = embed_results.text_ids.cpu().tolist()

        points = [
            PointStruct(id=id_, vector=emb, payload={"text": text, **payload})
            for id_, emb, text, payload in zip(ids, embeddings, texts, payloads)
        ]

        self.qdrant.upsert(collection_name=self.collection_name, points=points)
        return texts

    # -----------------------------
    # Query (semantic search)
    # -----------------------------
    def search(
        self,
        query: str | list[float],
        top_k: int = 5,
        with_payload: bool = True,
        score_threshold: float | None = None,
    ) -> QueryResponse:
        if isinstance(query, str):
            vector = self.embedder.embed([query]).embedding[0].cpu().tolist()
        elif isinstance(query, list):
            vector = query
        else:
            raise ValueError("Query must be a string or a list of floats.")

        results = self.qdrant.query_points(
            collection_name=self.collection_name,
            query=vector,
            limit=top_k,
            with_payload=with_payload,
            score_threshold=score_threshold,
        )

        return results

    def info(self) -> CollectionInfo:
        return self.qdrant.get_collection(self.collection_name)

    def clear(self):
        """Deletes all points in the collection."""
        self.qdrant.delete(
            collection_name=self.collection_name, points_selector={"filter": {}}
        )


In [4]:
from config import EMBEDDING_MODEL_PATH
from emcache import OllamaBackend

backend = OllamaBackend(base_url="http://127.0.0.1:11434")
storage = VectorStoreEmbedding(backend, collection_prefix="keywords")
# storage.embedder.load_state_dict(
#     torch.load(EMBEDDING_MODEL_PATH / f"{storage.collection_name}.pt")
# )

🧩 Deleting existing Qdrant collection: keywords_ollama_nomic-embed-text
🧩 Creating Qdrant collection: keywords_ollama_nomic-embed-text


In [11]:
storage.add_texts(["Hello World", "test text", "another example"])

['Hello World', 'test text', 'another example']

In [15]:
response = storage.search("hello World")
for res in response.points:
    print(f"Score: {res.score:<.4f} | Text: {res.payload['text']}")

Score: 1.0000 | Text: Hello World
Score: 0.3652 | Text: another example
Score: 0.3649 | Text: test text


In [ ]:
from emcache.huggingface import HuggingFaceBackend


# Initialize
backend = HuggingFaceBackend(model_name="heydariAI/persian-embeddings")
storage = VectorStoreEmbedding(backend, collection_prefix="test")


In [26]:
storage.add_texts(
    [
        "آیفون",
        "آیفون 11",
        "سوره حمد",
        "قرآن کریم",
        "هوش مصنوعی",
        "تولید محتوا",
        "طلا",
        "قیمت طلا",
        "ماشین",
        "خودرو",
        "بیتکوین",
        "ارز دیجیتال",
    ]
)

['آیفون',
 'آیفون 11',
 'سوره حمد',
 'قرآن کریم',
 'هوش مصنوعی',
 'تولید محتوا',
 'طلا',
 'قیمت طلا',
 'ماشین',
 'خودرو',
 'بیتکوین',
 'ارز دیجیتال']

In [ ]:
response = storage.search("bitcoin")
for res in response.points:
    print(f"Score: {res.score:<.4f} | Text: {res.payload['text']}")

Score: 1.0000 | Text: بیتکوین
Score: 0.2104 | Text: سوره حمد
Score: 0.2023 | Text: ارز دیجیتال
Score: 0.1916 | Text: قرآن کریم
Score: 0.1776 | Text: سلام دنیا


In [ ]:
import tqdm.auto as tqdm

points = list(storage.embed_keys.items())


batch_size = 128
batches = tqdm.trange(0, len(points), batch_size)

for i in batches:
    vectors = storage.embed_cache[i : i + batch_size].tolist()
    batch = [
        {"id": i, "vector": vectors[b_i], "payload": {"keyword": text}}
        for b_i, (text, i) in enumerate(points[i : i + batch_size])
    ]

    qdrant_client.upsert(collection_name="fa_keywords_custom", points=batch)


  0%|          | 0/452 [00:00<?, ?it/s]

In [33]:
from pathlib import Path


QUERY_PATH = Path("queries/website_by_keyword.sql")
WEBSITE_QUERY = QUERY_PATH.read_text(encoding="utf-8")

query = "خرید هارد"


results = storage.search(query=query, top_k=256)

related_keywords = []
for r in results.points:
    related_keywords.append(r.payload["text"])
    print(r.payload["text"], "→", r.score)


scores = [r.score for r in results.points]
related_keywords = [r.payload["text"] for r in results.points]

df = ch_client.query_df(
    WEBSITE_QUERY,
    parameters={"keywords": related_keywords, "scores": scores},
)

df
df.groupby("website").count().sort_values("keyword", ascending=False)

خودرو → 0.23558846
قیمت طلا → 0.22979479
ارز دیجیتال → 0.22432944
آیفون → 0.20793667
طلا → 0.20032229
ماشین → 0.19134352
آیفون 11 → 0.18514821
هوش مصنوعی در سئو → 0.17690553
سلام دنیا → 0.16192293
هوش مصنوعی → 0.14489606
سوره حمد → 0.13165203
بیتکوین → 0.1200002
تولید محتوا → 0.079289205
قرآن کریم → 0.062037934
نیک نویس پلتفرم تولید محتوا → 0.0032443386


,keyword,score,average_position
website,,,
,5,5,5
www.zoomit.ir,5,5,5
diginoy.com,4,4,4
www.asriran.com,4,4,4
www.khabaronline.ir,4,4,4
www.irna.ir,3,3,3
www.tgju.org,3,3,3
www.tabnak.ir,3,3,3
arzdigital.com,2,2,2


In [ ]:
debug_query = """
WITH
    {keywords:Array(String)} AS keywords,
    {scores:Array(Float32)} AS scores
SELECT
    q.keyword,
    q.score,
    d.website,
    d.average_position
FROM
(
    SELECT
        arrayJoin(arrayEnumerate(keywords)) AS idx,
        keywords[idx] AS keyword,
        scores[idx] AS score
) AS q
LEFT OUTER JOIN
(
    SELECT
        keyword,
        website,
        average_position
    FROM ahrefs.keywords
) AS d
ON q.keyword = d.keyword
"""

scores = [r.score for r in results.points]
related_keywords = [r.payload["keyword"] for r in results.points]

df_debug = ch_client.query_df(
    debug_query, parameters={"keywords": related_keywords, "scores": scores}
)
print(df_debug)


             keyword     score                 website  average_position
0          ایده تولد  0.946327          www.chetor.com          2.750000
1              توحید  0.937184        www.beytoote.com          7.000000
2         سوره توحید  0.894493        www.beytoote.com          2.666667
3         سوره توحید  0.894493         www.delgarm.com         14.000000
4       ماه های تولد  0.891508  www.tasvirezendegi.com          4.000000
..               ...       ...                     ...               ...
367  تولد برای خواهر  0.813686  www.tasvirezendegi.com          1.000000
368     احادیث کوتاه  0.813636                  ana.ir         52.000000
369     احادیث کوتاه  0.813636           www.talab.org          6.000000
370     شایلی محمودی  0.813529            persianv.com         10.000000
371     شایلی محمودی  0.813529       www.niksalehi.com          5.000000

[372 rows x 4 columns]


In [ ]:
query = """
SELECT
    website,
    sum(average_position * volume) / sum(volume) AS weighted_avg_position
FROM ahrefs.keywords
WHERE keyword IN {keywords:Array(String)}
GROUP BY website
ORDER BY weighted_avg_position ASC
LIMIT 20
"""

df = ch_client.query_df(query, parameters={"keywords": related_keywords})
df

,website,weighted_avg_position
0,www.zoomit.ir,1.000000
1,www.tgju.org,2.684793
2,digiato.com,4.406250
3,arzdigital.com,6.445607
4,cryptonegar.com,6.734848
5,www.kojaro.com,6.746941
6,www.beytoote.com,6.893842
7,safarpin.com,7.000000
8,www.chetor.com,7.741090
9,www.asriran.com,9.616824
